In [1]:
import pandas as pd

genes = pd.read_csv("../out_data/important_roary_out/gene_presence_absence.csv.gz", compression="gzip",low_memory=False)
COLS_KEEP = ["Gene","Annotation","No. isolates"]
genes = genes.loc[:,COLS_KEEP]
genes['hypothetical'] = genes['Annotation'].str.contains("hypothetical")
genes = genes.loc[genes['No. isolates'] == 60,:]
genes = genes.drop(columns=['No. isolates'])
genes


,Gene,Annotation,hypothetical
0,group_1112,hypothetical protein,True
1,fepE,Ferric enterobactin transport protein FepE,False
2,etfB,Electron transfer flavoprotein subunit beta,False
3,lacZ,Beta-galactosidase,False
4,alkA,DNA-3-methyladenine glycosylase 2,False
...,...,...,...
2444,cof,HMP-PP phosphatase,False
2445,fdhD,Sulfur carrier protein FdhD,False
2446,cueO,Blue copper oxidase CueO,False
2447,rlmI,Ribosomal RNA large subunit methyltransferase I,False


In [2]:
synon = pd.read_csv("../in_data/synom_names.txt", sep="\t")
synon[["b_num","jw_num"]] = synon['Ordered locus name'].str.split(";", expand=True)
other_names = synon['Gene name and synonyms'].str.split("\s*;\s*", expand=True)
synon[["synon_"+str(_) for _ in range(len(other_names.columns))]] = other_names
synon

,Ordered locus name,Swiss-Prot Entry name,AC,EcoGene accession number,Length,Gene name and synonyms,b_num,jw_num,synon_0,synon_1,synon_2,synon_3,synon_4,synon_5,synon_6,synon_7,synon_8
0,b0001; JW4367,LPT_ECOLI,P0AD86,EG11277,21,thrL,b0001,JW4367,thrL,None,None,None,None,None,None,None,None
1,b0002; JW0001,AK1H_ECOLI,P00561,EG10998,820,thrA; thrA1; thrA2,b0002,JW0001,thrA,thrA1,thrA2,None,None,None,None,None,None
2,b0003; JW0002,KHSE_ECOLI,P00547,EG10999,310,thrB,b0003,JW0002,thrB,None,None,None,None,None,None,None,None
3,b0004; JW0003,THRC_ECOLI,P00934,EG11000,428,thrC,b0004,JW0003,thrC,None,None,None,None,None,None,None,None
4,b0005; JW0004,YAAX_ECOLI,P75616,EG14384,98,yaaX,b0005,JW0004,yaaX,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4302,b4703; JW4072.1,PMRR_ECOLI,G3MTW7,EG14499,29,pmrR,b4703,JW4072.1,pmrR,None,None,None,None,None,None,None,None
4303,b4705; JW0800.1,MNTS_ECOLI,P0DKB3,EG14500,42,mntS; rybA,b4705,JW0800.1,mntS,rybA,None,None,None,None,None,None,None
4304,b4706,IROK_ECOLI,U3PVA8,EG14501,21,iroK,b4706,None,iroK,None,None,None,None,None,None,None,None
4305,b4715,YTIA_ECOLI,P0DN74,-,85,ytiA,b4715,None,ytiA,None,None,None,None,None,None,None,None


In [152]:
# this is a pretty convuluted way to check if there are any best matches for merging that have synonyms 
# that are found multiple times in the table
# takes the first occuring use of the synonym to use as the canonical name
# however, this only adds 6 extra genes to the list, so it's not worth it IMO

indices = []
for index in genes.index:
    gene = genes.loc[index]["Gene"]
    index_list = other_names.loc[other_names.isin([gene]).any(axis=1)].index.tolist()
    if len(index_list) == 0:
        continue
    if len(index_list) > 1:
        # gets first appearance of gene
        found_index = other_names.loc[index_list].isin([gene]).idxmax(axis=1).idxmin()
        # print(found_index) #finds 18 genes that have this issue, only 6 are really added in the end
    else:
        found_index = index_list[0]
    indices.append((index,found_index))
len(indices)


1886

In [5]:
core = genes.merge(synon, left_on="Gene", right_on="synon_0", how="inner").dropna(axis=1, how="all")
core = core.drop(columns=['hypothetical'])
core = core.drop(columns=['synon_'+str(_) for _ in range(0,9)])
core = core.drop(columns=['Ordered locus name'])

#core.to_csv("../out_data/core_genes.csv", index=False)

In [ ]:
# trying to scrape from this paper: https://www.pnas.org/doi/full/10.1073/pnas.1105168108#supplementary-materials
# couldn't do it in the end

# data_loc = "https://www.pnas.org/doi/suppl/10.1073/pnas.1105168108/suppl_file/sd01.txt"

with open(data_loc) as fh:
    paper = fh.read()
paper = paper.split("\n")[2:-1]
paper = [_.split("\t") for _ in paper]


# all kinds of bespoke cleaning needed
# maybe a better way to do this, but the data isn't useful in the end
papers = []
cur_phylo = None
skip = False
for line in paper:
    if "Phylogroup" in line[0]:
        cur_phylo = line
        skip = True
    else:
        if skip == True:
            skip = False
            continue
        else:   
            papers.append(cur_phylo + line)
        
papers = pd.DataFrame(papers, columns=["Phylogroup","NCBI_GENOME.Gene_Name.Gene_ID.","Hit Count"])
papers = papers.loc[pd.to_numeric(papers['Hit Count'], errors='coerce').notnull(),:]
papers['Hit Count'] = pd.to_numeric(papers['Hit Count'])
papers

In [ ]:
papers['NCBI_GENOME.Gene_Name.Gene_ID.'].str.split(".", expand=True)[0].value_counts().hist()